In [2]:
import pandas as pd

# Read CSV Files
df_errors = pd.read_csv('PdM_errors.csv')
df_failures = pd.read_csv('PdM_failures.csv')
df_machines = pd.read_csv('PdM_machines.csv')
df_maint = pd.read_csv('PdM_maint.csv')
df_telemetry = pd.read_csv('PdM_telemetry.csv')

# Convert datetime columns to datetime objects for accurate merging and manipulation
df_errors['datetime'] = pd.to_datetime(df_errors['datetime'])
df_failures['datetime'] = pd.to_datetime(df_failures['datetime'])
df_maint['datetime'] = pd.to_datetime(df_maint['datetime'])
df_telemetry['datetime'] = pd.to_datetime(df_telemetry['datetime'])

# Aggregate telemetry data to daily averages to match the frequency of maintenance and failure records
df_telemetry_daily = df_telemetry.groupby([df_telemetry['datetime'].dt.date, 'machineID']).mean().reset_index()
df_telemetry_daily['datetime'] = pd.to_datetime(df_telemetry_daily['datetime'])

# error indicators (binary columns for error 1 to 5)
for i in range(1, 6):
    df_errors[f'error{i}'] = (df_errors['errorID'] == f'error{i}').astype(int)

# Summarize errors daily by machineID
df_errors_daily = df_errors.groupby([df_errors['datetime'].dt.date, 'machineID'])[f'error1', 'error2', 'error3', 'error4', 'error5'].sum().reset_index()
df_errors_daily['datetime'] = pd.to_datetime(df_errors_daily['datetime'])

# Creates failure indicators (binary columns for comp1_fail to comp4_fail)
for i in range(1, 5):
    df_failures[f'comp{i}_fail'] = (df_failures['failure'] == f'comp{i}').astype(int)

# Summarize failures daily by machineID
df_failures_daily = df_failures.groupby([df_failures['datetime'].dt.date, 'machineID'])[f'comp1_fail', 'comp2_fail', 'comp3_fail', 'comp4_fail'].sum().reset_index()
df_failures_daily['datetime'] = pd.to_datetime(df_failures_daily['datetime'])

# Creates maintenance indicators (binary columns for comp1_maint to comp4_maint)
for i in range(1, 5):
    df_maint[f'comp{i}_maint'] = (df_maint['comp'] == f'comp{i}').astype(int)

# Summarize maintenance daily by machineID
df_maint_daily = df_maint.groupby([df_maint['datetime'].dt.date, 'machineID'])[f'comp1_maint', 'comp2_maint', 'comp3_maint', 'comp4_maint'].sum().reset_index()
df_maint_daily['datetime'] = pd.to_datetime(df_maint_daily['datetime'])

# Merge processed DataFrames
df_final = df_telemetry_daily.merge(df_machines, on='machineID', how='left')
df_final = df_final.merge(df_errors_daily, on=['datetime', 'machineID'], how='left').fillna(0)
df_final = df_final.merge(df_failures_daily, on=['datetime', 'machineID'], how='left').fillna(0)
df_final = df_final.merge(df_maint_daily, on=['datetime', 'machineID'], how='left').fillna(0)

# Ensure the final DataFrame includes all required columns
required_columns = [
    'machineID', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration',
    'error1', 'error2', 'error3', 'error4', 'error5',
    'comp1_fail', 'comp2_fail', 'comp3_fail', 'comp4_fail',
    'comp1_maint', 'comp2_maint', 'comp3_maint', 'comp4_maint'
]

# Reorder and filter the columns to match the requirement
df_final = df_final[required_columns]

# Save the final DataFrame
output_path = 'final_dataframe.csv'
df_final.to_csv(output_path, index=False)

<ipython-input-2-610b9730a35f>:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_telemetry_daily = df_telemetry.groupby([df_telemetry['datetime'].dt.date, 'machineID']).mean().reset_index()
<ipython-input-2-610b9730a35f>:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_errors_daily = df_errors.groupby([df_errors['datetime'].dt.date, 'machineID'])[f'error1', 'error2', 'error3', 'error4', 'error5'].sum().reset_index()
<ipython-input-2-610b9730a35f>:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_failures_daily = df_failures.groupby([df_failures['datetime'].dt.date, 'machineID'])[f'comp1_fail', 'comp2_fail', 'comp3_fail', 'comp4